<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# uncomment on google colab, comment on kaggle.

In [1]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterer-preproc

Saving kaggle.json to kaggle.json
 99% 204M/206M [00:10<00:00, 23.5MB/s]
100% 206M/206M [00:10<00:00, 20.9MB/s]


In [2]:
# !unzip /content/betterer-preproc.zip

In [3]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

100% 34.0M/34.1M [00:02<00:00, 23.9MB/s]
100% 34.1M/34.1M [00:02<00:00, 14.1MB/s]


In [4]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem207.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem208.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem209.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem21.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem210.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem211.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem212.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem213.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem214.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem215.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem216.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem217.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem218.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/m

# PRE PROCESS
.JPG TO .NPY

In [7]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/content/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [8]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [9]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


5121

In [10]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 52 2560 1792


In [11]:
import imgaug.augmenters as iaa

# Define the augmenter
augmenter = iaa.Sequential([
    iaa.Affine(rotate=(-10, 10)),  # Rotate the image by -10 to 10 degrees
    iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur with a sigma ranging from 0.0 to 1.0
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05 times 255
    iaa.Multiply((0.9, 1.1)),
    iaa.LinearContrast((0.9, 1.1)),
])

augmented_images = []

for path in moderateDem:
    path = os.path.join("/kaggle/working/train", path)
    image = np.load(path)

    for _ in range(35):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [12]:
len(augmented_images)

1820

In [13]:
import cv2
import os


output_directory = "/kaggle/working/train"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
    np.save(output_path, image)


print("Augmented images saved successfully.")


Augmented images saved successfully.


In [14]:
augmented_images = []

for path in mildDem:
    path = os.path.join("/kaggle/working/train", path)
    image = np.load(path)

    for _ in range(2):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [15]:
len(augmented_images)

1434

In [16]:
output_directory = "/kaggle/working/train"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"mildDem{i+718}.npy")
    np.save(output_path, image)


print("Augmented images saved successfully.")

Augmented images saved successfully.


In [17]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

8375

In [18]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2151 1872 2560 1792


In [19]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
2    2560
0    2151
1    1872
3    1792
Name: count, dtype: int64

In [20]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [21]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [22]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[2 3 0 ... 2 0 3]
[2 3 1 ... 0 1 2]


In [23]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(6700, 208, 176, 3) (1675, 208, 176, 3) (6700,) (1675,)


In [24]:
print(test_labels)
test_labels_encoded

[2 3 1 ... 0 1 2]


array([2, 3, 1, ..., 0, 1, 2])

In [25]:
print(x.index, x.values, x.array)

Index([2, 0, 1, 3], dtype='int64', name='Class') [2560 2151 1872 1792] <PandasArray>
[2560, 2151, 1872, 1792]
Length: 4, dtype: int64


In [26]:
s = x / len(files)
class_weights = {i:j for i, j in zip(s.index, s.values)}

class_weights
# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


{2: 0.3056716417910448,
 0: 0.25683582089552237,
 1: 0.22352238805970148,
 3: 0.21397014925373134}

In [43]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

def conv_block(filters, act='relu'):
    """Defining a Convolutional NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D((2, 2)))

    return block

In [44]:
def dense_block(units, dropout_rate, act='relu'):
    """Defining a Dense NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))

    return block

In [38]:
!pip install tensorflow_addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.9 MB/s eta 0:00:00


In [45]:
import tensorflow as tf
import tensorflow_addons as tfa
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           tfa.metrics.F1Score(num_classes=4)]

In [46]:
input_shape = train_images[0].shape

train_labels_encoded = to_categorical(train_labels, num_classes=4)
test_labels_encoded = to_categorical(test_labels, num_classes=4)

sample_weights = compute_sample_weight(class_weight='balanced', y=np.argmax(train_labels_encoded, axis=1))

In [47]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """

    model = Sequential([
        Input(shape=(input_shape)),
        Conv2D(16, 3, activation=act, padding='same'),
        Conv2D(16, 3, activation=act, padding='same'),
        MaxPooling2D((2, 2)),
        conv_block(32),
        conv_block(64),
        conv_block(128),
        Dropout(0.2),
        conv_block(256),
        Dropout(0.2),
        Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        Dense(4, activation='softmax')
    ], name = "cnn_model")

    return model

In [71]:
model = construct_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

model.summary()


Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 208, 176, 16)      448       
                                                                 
 conv2d_19 (Conv2D)          (None, 208, 176, 16)      2320      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 104, 88, 16)       0         
 ng2D)                                                           
                                                                 
 sequential_9 (Sequential)   (None, 52, 44, 32)        14016     
                                                                 
 sequential_10 (Sequential)  (None, 26, 22, 64)        55680     
                                                                 
 sequential_11 (Sequential)  (None, 13, 11, 128)       221952    
                                                         

In [72]:
from keras.optimizers import Adam
def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [73]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc') > 0.995:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True

my_callback = MyCallback()

CALLBACKS = [my_callback, lr_scheduler]

In [74]:
EPOCHS = 50

history = model.fit(train_images, train_labels_encoded, validation_data=(test_images, test_labels_encoded), callbacks=CALLBACKS, epochs=EPOCHS)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
  6/210 [..............................] - ETA: 18s - loss: 2.2209 - acc: 0.6186 - auc: 0.8348 - f1_score: 0.4455

210/210 [==============================] - 25s 87ms/step - loss: 1.3041 - acc: 0.4928 - auc: 0.7612 - f1_score: 0.4740 - val_loss: 2.0422 - val_acc: 0.3672 - val_auc: 0.6383 - val_f1_score: 0.2992 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
210/210 [==============================] - 17s 81ms/step - loss: 0.9014 - acc: 0.5797 - auc: 0.8528 - f1_score: 0.5730 - val_loss: 2.4661 - val_acc: 0.4101 - val_auc: 0.6495 - val_f1_score: 0.3435 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
210/210 [==============================] - 17s 81ms/step - loss: 0.6899 - acc: 0.6825 - auc: 0.9127 - f1_score: 0.6769 - val_loss: 1.5833 - val_acc: 0.4621 - val_auc: 0.7925 - val_f1_score: 0.2870 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
210/210 [==============================] - 18s 85ms/step - loss: 0.6224 - acc: 0.7157 - auc: 0.92

In [28]:


# # Assuming train_images and test_images are your input image data


# model = Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4, activation='softmax'))


In [ ]:
# import math
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_images, train_labels_encoded,
#                     steps_per_epoch = int(len(train_images) / 32),
#                     epochs=50,
#                     validation_data=(test_images, test_labels_encoded),
#                     sample_weight=sample_weights,
#                     callbacks=[lr_scheduler])

In [52]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)
y_prob = tf.nn.softmax(y_prob).numpy()

# Compute ROC AUC for each class
roc_auc = roc_auc_score(test_labels_encoded, y_prob, average='macro')

print("ROC AUC:", roc_auc)

53/53 [==============================] - 1s 14ms/step
ROC AUC: 0.9982698320706782


In [53]:
# Save the model
model.save("my_cnn_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [54]:
import os
import numpy as np
directory = '/kaggle/working/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1279

In [55]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [56]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [57]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [58]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=4)
# Verify the encoded labels
print(labels.shape)

(1279, 4)


In [59]:
l = []
for i in labels:
  for j in range(4):
    if i[j] == 1:
      l.append(j)


In [75]:
y_prob = model.predict(images)

# y_prob = tf.nn.softmax(y_prob).numpy()
# Get the class with the highest probability for each prediction
y_prob = np.argmax(y_prob, axis=1)  # Argmax gives index of max value
# for i in range(len(labels)):
#   print(l[i], y_prob[i])

40/40 [==============================] - 1s 15ms/step


In [76]:
from sklearn.metrics import classification_report

# Assuming you have predicted labels (predicted_classes) and true labels (test_labels_encoded)
report = classification_report(l, y_prob, output_dict=True)
for i in report:
  print(i, report[i])

0 {'precision': 0.9743589743589743, 'recall': 0.2122905027932961, 'f1-score': 0.3486238532110092, 'support': 179}
1 {'precision': 1.0, 'recall': 0.5833333333333334, 'f1-score': 0.7368421052631579, 'support': 12}
2 {'precision': 0.9041916167664671, 'recall': 0.471875, 'f1-score': 0.620123203285421, 'support': 640}
3 {'precision': 0.4860956618464961, 'recall': 0.9754464285714286, 'f1-score': 0.6488492947290274, 'support': 448}
accuracy 0.6129788897576232
macro avg {'precision': 0.8411615632429844, 'recall': 0.5607363161745145, 'f1-score': 0.588609614122154, 'support': 1279}
weighted avg {'precision': 0.7684626643065094, 'recall': 0.6129788897576232, 'f1-score': 0.5932831189438642, 'support': 1279}


In [77]:
model.evaluate(images, labels)

40/40 [==============================] - 1s 15ms/step - loss: 2.2908 - acc: 0.6130 - auc: 0.8029 - f1_score: 0.5886


[2.29081130027771,
 0.6129788756370544,
 0.8029040694236755,
 array([0.34862384, 0.7368421 , 0.6201232 , 0.6488493 ], dtype=float32)]

In [78]:

import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(images)
y_prob = tf.nn.softmax(y_prob).numpy()

# Compute ROC AUC for each class
roc_auc = roc_auc_score(labels, y_prob, average='macro')

print("ROC AUC:", roc_auc)

40/40 [==============================] - 1s 13ms/step
ROC AUC: 0.8089602223044418
